# マルコフ連鎖モンテカルロ法

ベクトルや行列を太文字表記していないので要注意

In [1]:
# 必要なライブラリをインポート
from tqdm import trange
import numpy as np
import scipy.stats as st
import pandas as pd
import pymc3 as pm
import matplotlib.pyplot as plt
import japanize_matplotlib
%matplotlib inline

import warnings
warnings.simplefilter('ignore')

## マルコフ連鎖と不変分布

### マルコフ連鎖の概要
確率変数の系列$\{X_t\}_{t=0}^{\infty}$に対して，$\{X_s\}_{s=0}^{t-1}$の実現値$\{x_s\}_{s=0}^{t-1}$が与えられた下で$X_t$が部分集合$A \subseteq \chi$内に値をとる条件付き確率が以下のように表現されるとき，$\{X_t\}_{t=0}^{\infty}$は**マルコフ連鎖**であるという．
$$
    Pr\{ X_t \in A| X_0=x_0, \dotsc , X_{t-1}=x_{t=1} \} = 
    Pr\{ X_t \in A| X_{t-1}=x_{t-1} \} \tag{1-1}
$$
> 要は，確率変数$X_t$の条件付き確率分布は直近の確率変数の実現値$x_{t-1}$にのみ依存している．

マルコフ連鎖における重要な概念
- 斉時性: (1-1)式の条件付き確率が時点$t$に依存しないこと
- 規約性: 確率分布$f$で実現しうる$\chi$内のいかなる場所にもマルコフ連鎖が到達可能であること
- 周期性: 一定の時間を経過しないとマルコフ連鎖が再び訪れることができない場所が$\chi$内に存在すること
- 再帰性: 確率分布$f$で実現しうる$\chi$内のいかなる場所にもマルコフ連鎖が有限時間内に必ず訪れること

$X_t$の$\{x_s\}_{s=0}^{t-1}$が与えられた下での条件付き確率分布は以下の通り．
$$
    f_t(x_t|x_0, \dotsc, x_{t-1}) = f(x_t|t_{t-1}) \tag{1-2}
$$
マルコフ連鎖が斉時的であるとき，(1-2)式の右辺の関数形は時点$t$に依存せず，$\{X_s\}_{s=0}^t$のどう次確率分布は以下のように与えられる．
$$
    f(x_0, \dotsc, x_t) = 
    f_0(x_0) \prod_{s=1}^t f(x_s|x_{s-1}) \tag{1-3}
$$
この$f(x_t|x_{t-1})$をマルコフ連鎖の**遷移核**と呼ぶ．

### マルコフ連鎖の不変分布
以下を満たす$\bar{f}$をマルコフ連鎖の**不変分布**または**定常分布**という．
$$
    \bar{f}(\tilde{x}) = 
    \int_{\chi} \bar{f}(x)K(x, \tilde{x})dx, 
    あるいは \
    \bar{f} = \bar{f} \circ K, \tag{1-4}
$$
ここで$K$はマルコフ連鎖の遷移核を表す．


### マルコフ連鎖からの乱数系列の生成
斉時的なマルコフ連鎖は$f_0$と$K$にのみ依存するという簡単な構造をしており，$f_0$と$K$からの乱数生成が可能であれば，マルコフ連鎖からの乱数生成を以下の手順で行うことができる．
1. $t=1$として$\tilde{x}_0 \gets f_0(x_0)$．
2. $\tilde{x}_t \gets K(\tilde{x}_{t-1}, x_t)$．
3. $t$を$1$増やして手順2に戻る

マルコフ連鎖が$\bar{f}$に収束したと判断されるまで乱数$\{\tilde{x_t}\}_{t=0}^{t^*}$を生成し続けることを**検査稼働期間**（バーンイン）と呼ぶ．そして，$\bar{f}$に収束したと判断した回以降のマルコフ連鎖から生成した$\{\tilde{x}_t\}_{t={t^*}+1}^{{t^*}+T}$は$\bar{f}$からのモンテカルロ標本として利用可能である．

不変分布への収束後のサンプリングは大きく分けて(i)多重連鎖法，(ii)単一連鎖法 の2通り

(i)多重連鎖法の手順
1. $\tilde{x}_0 \gets f_0(x_0)$．
2. マルコフ連鎖から分布が収束するまで乱数を生成する．
3. 最後の$\tilde{x}_t$を保存し，手順1に戻る．

(ii)単一連鎖法の手順
1. $\bar{f}$の取りうる領域から$\tilde{x}_0$を選ぶ．
2. マルコフ連鎖から分布が収束するまで乱数を生成する．
3. 必要な数の$\tilde{x}_t$を続けてマルコフ連鎖から生成する．

## メトロポリス-ヘイスティングズ・アルゴリズム（M-Hアルゴリズム）
マルコフ連鎖サンプリング法のために事後分布を不変分布に持つマルコフ連鎖を構築する方法の1つ．

$\theta$を確率変数，$f(\theta)$を$\theta$の乱数を生成したい目標分布とし，$f$において$\theta$の取りうる値の集合$\Theta \in \{ \theta: f(\theta)>0 \}$を考える．

以下の仮定をおく．
- $f$から直接$\theta$の乱数を生成できないが，代わりにマルコフ連鎖の遷移核$q(\varphi, \theta)$からは乱数を容易に生成できる
- 遷移核$q$は斉時的である
- 任意のペア$(\varphi, \theta) \in \Theta \times \theta$に対して，$q(\varphi, \theta)>0$

このとき，$f$と$q$をうまく組み合わせて$f$に収束するマルコフ連鎖を作るテクニックを**M-Hアルゴリズム**と呼ぶ．ベイズ統計学への応用という意味では，**目標分布$f$の基準か定数が未知であっても問題なく適用できる**点で極めて有用．

M-Hアルゴリズムの手順
1. $t=1$として初期値$\theta^{(0)}$を設定する．
2. $\theta^(t)$の候補$\tilde{\theta}$をマルコフ連鎖から生成する．
$$
    \tilde{\theta} \gets q(\theta^{(t-1)}, \theta).
$$
3. $\tilde{\theta}$の採択確率を以下の計算式で計算する．
$$
    \alpha(\theta^{(t-1)}, \tilde{\theta}) = 
    \min \bigg\{ 
        \frac{f(\tilde{\theta})q(\tilde{\theta}, \theta^{(t-1)})}
            {f(\theta^{(t-1)})q(\theta^{(t-1)}, \tilde{\theta})}
        , 1 \bigg\}. \tag{2-1}
$$
4. \theta^{(t)}を以下のルールに従い更新する．
$$
    \tilde{u} \gets U(0, 1), \ 
    \theta^{(t)} = 
        \begin{cases}
            \tilde{\theta}, & (\tilde{u} \leq \alpha(\theta^{(t-1)}, \tilde{\theta})), \\
            \theta^{(t-1)}, & (\tilde{u} > \alpha (\theta^{(t-1)}, \tilde{\theta})).
        \end{cases}
$$
5. $t$を$1$つ増やして手順2に戻る．

M-Hアルゴリズムは$q$の選択に応じて幾つかに分類され，以下の手法などがある．
1. ランダムウォーク連鎖
2. 独立連鎖
3. ハミルトニアン・モンテカルロ（HMC）法

### ランダムウォーク連鎖
遷移核$q$として，以下のランダムウォーク過程を使用するM-Hアルゴリズム．
$$
    \theta = \varphi + \epsilon, \ 
    E[\epsilon] = 0, \ 
    Var[\epsilon] = \Sigma, 
    \tag{2-2}
$$

ランダムウォーク連鎖の手順
1. $t=1$として初期値$\theta^{(0)}$を設定する．
2. $\theta{(t)}$の候補$\tilde{\theta}$をランダムウォーク連鎖(2-2)から生成する．
3. $\tilde{\theta}$の採択確率を以下の式で計算する．
$$
    \alpha(\theta^{t-1}, \tilde{\theta}) = 
    \min \bigg\{ \frac{f(\theta)}{f(\theta^{t-1})}, 1 \bigg\}. 
    \tag{2-3}
$$
4. $\theta^{(t)}$を以下のルールに従い更新する．
$$
    \tilde{u} \gets U(0, 1), \ 
    \theta^{(t)} = 
        \begin{cases}
            \tilde{\theta}, & (\tilde{u} \leq \alpha(\theta^{(t-1)}, \tilde{\theta})), \\
            \theta^{(t-1)}, & (\tilde{u} > \alpha (\theta^{(t-1)}, \tilde{\theta})).
        \end{cases}
$$
5. $t$を$1$つ増やして手順2に戻る．

### 独立連鎖
遷移核$q$から生成される$\theta^{(t)}$を$\theta^{(t-1)}$と独立に生成するM-Hアルゴリズムの派生形．

独立連鎖の手順
1. $t=1$として初期値$\theta^{(0)}$を設定する．
2. $\theta^{(t)}$の候補$\tilde{\theta}$を提案分布$g(\theta)$から生成する．
$$
    \tilde{\theta} \gets g(\theta).
$$
3. $\tilde{\theta}$の採択確率を以下の式で計算する．
$$
    \alpha(\theta^{(t-1)}, \tilde{\theta}) = 
    \min \bigg\{ 
        \frac{f(\tilde{\theta})g(\theta^{(t-1)})}
        {f(\theta^{(t-1)})g(\tilde{\theta})}
        , 1
    \bigg\}. 
    \tag{2-4}
$$
4. $\theta^{(t)}$を以下のルールに従い更新する．
$$
    \tilde{u} \gets U(0, 1), \ 
    \theta^{(t)} = 
        \begin{cases}
            \tilde{\theta}, & (\tilde{u} \leq \alpha(\theta^{(t-1)}, \tilde{\theta})), \\
            \theta^{(t-1)}, & (\tilde{u} > \alpha (\theta^{(t-1)}, \tilde{\theta})).
        \end{cases}
$$
5. $t$を$1$つ増やして手順2に戻る．


### ハミルトニアン・モンテカルロ（HMC）法
$\theta$と同じ次元の$\zeta$という補助的確率変数の確率密度関数を$g(\zeta)$とし，この$(\theta, \zeta)$に対してマルコフ連鎖$q((\theta, \zeta), (\tilde{\theta}, \tilde{\zeta}))$を考える．

HMC法の手順
1. $t=1$として初期値$\theta^{(0)}$を設定する．
2. $\zeta_0 \gets N_i(0, \Sigma)$.
3. $j=0$として$\zeta \gets \zeta_0. \ \theta \gets \theta^{(t-1)}.$
4. 以下の通りにデータを生成する
$$
    \zeta^* \gets \zeta + \frac{\epsilon}{2} \nabla_{\theta}\log f(\theta), \\ 
    \tilde{\theta} \gets \theta + \epsilon \Sigma^{-1} \zeta^*, \\ 
    \tilde{\zeta} \gets \zeta^* + \frac{\epsilon}{2} \nabla_{\theta}\log f(\theta).
$$
5. $j$を$1$つ増やし，$j<L$ならば$\zeta \gets \tilde{\zeta}, \ \theta \gets \tilde{\theta}$として，手順4に戻る．
6. $\tilde{\theta}$の採択確率を以下の式で計算する．
$$
    \alpha(\theta^{t-1}, \tilde{\theta}) = 
    \min \bigg\{ 
        \frac{\exp \Big[ \log f(\tilde{\theta}) - 
                \frac{1}{2} \tilde{\zeta}^T \Sigma^{-1} \tilde{\zeta} \Big]}
            {\exp \Big[ \log f(\theta^{(t-1)}) 
                \frac{1}{2} \zeta_0^T \Sigma^{-1} \zeta_0 \Big]}
        , 1
    \bigg\}.
$$
7. $\theta^{(t)}$を以下のルールに従い更新する．
$$
    \tilde{u} \gets U(0, 1), \ 
    \theta^{(t)} = 
        \begin{cases}
            \tilde{\theta}, & (\tilde{u} \leq \alpha(\theta^{(t-1)}, \tilde{\theta})), \\
            \theta^{(t-1)}, & (\tilde{u} > \alpha (\theta^{(t-1)}, \tilde{\theta})).
        \end{cases}
$$
8. $t$を$1$つ増やして手順2に戻る．

## ギブズ・サンプラー
マルコフ連鎖サンプリング法のために事後分布を不変分布に持つマルコフ連鎖を構築する方法の1つ．